# **1. Installation**

In [ ]:
!pip install google-generativeai
!pip install transformers
!pip install tika
!pip install PyPDF2
!pip install reportlab
!pip install googletrans==4.0.0-rc1


# **2. Generate Multilingual Content**

In [ ]:
import PyPDF2
import google.generativeai as genai
from io import BytesIO
import requests
import multiprocessing
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import google.generativeai as genai
from google.colab import userdata
import warnings
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
import markdown
import time

warnings.filterwarnings("ignore")

# Configure the Google Generative AI API
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

def split_text_into_chunks(text, max_chunk_size=2000):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 > max_chunk_size:
            chunks.append(current_chunk)
            current_chunk = sentence
        else:
            if current_chunk:
                current_chunk += ". " + sentence
            else:
                current_chunk = sentence

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

def generate_content(prompt, max_tokens=1024, retries=3, wait=5):
    model = genai.GenerativeModel('gemini-pro')
    for attempt in range(retries):
        try:
            response = model.generate_content(prompt)
            if response.parts:
                return response.text.strip()
            else:
                raise ValueError("No valid parts in response.")
        except Exception as e:
            print(f"Error: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {wait} seconds...")
                time.sleep(wait)
            else:
                return f"Failed to generate content after {retries} attempts."

def translate_text(text, language_name):
    model = genai.GenerativeModel('gemini-pro')
    prompt = f"Translate the following text to {language_name}: {text}"
    response = model.generate_content(prompt)
    if response.parts:
        return response.text.strip()
    else:
        raise ValueError("No valid parts in response.")

def create_prompts(text, task_type):
    prompts = {
        "mcq": f"Read the following text carefully and generate multiple-choice questions. Each question should include:\n"
               f"1. A clear and concise question based on the text.\n"
               f"2. Four answer options (A, B, C, D), with one correct answer clearly indicated.\n"
               f"3. The questions should cover key concepts, definitions, critical points, and significant details discussed in the text.\n"
               f"4. Ensure the options are plausible and relevant to the content.\n\n"
               f"Text:\n{text}\n\nMCQ:",
        "fill_in_the_blank": f"Read the following text thoroughly and generate fill-in-the-blank questions. Each question should include:\n"
                            f"1. A sentence from the text with one key term or concept replaced by a blank.\n"
                            f"2. The correct term or concept that completes the sentence accurately.\n"
                            f"3. Focus on important information, such as key terms, dates, names, and concepts that are critical to understanding the text.\n\n"
                            f"Text:\n{text}\n\nFill in the blank:",
        "short_answer": f"Read the following text attentively and generate short answer questions. Each question should include:\n"
                        f"1. A clear and specific question that requires a brief response.\n"
                        f"2. The response should address key points, explanations, or definitions provided in the text.\n"
                        f"3. Ensure the questions encourage critical thinking and comprehension of the material, focusing on important details and concepts.\n\n"
                        f"Text:\n{text}\n\nShort answer question:",
        "course": f"Read the following text and generate a comprehensive, structured curriculum content. The content should include:\n"
                  f"1. Learning objectives and outcomes.\n"
                  f"2. Topic-wise breakdown with detailed descriptions.\n"
                  f"3. Key concepts, definitions, and explanations.\n"
                  f"4. Examples, illustrations, and case studies.\n"
                  f"5. Assessment and evaluation criteria.\n\n"
                  f"Text:\n{text}\n\nCurriculum Content:",
    }
    return prompts.get(task_type, "")

def get_user_input(prompt):
    return input(prompt)

def chatbot():
    pdf_path = get_user_input("Enter the path to your PDF file: ")
    pdf_text = extract_text_from_pdf(pdf_path)

    languages = [
        "Arabic", "Czech", "German", "English", "Spanish", "Estonian", "Finnish", "French", "Gujarati",
        "Hindi", "Italian", "Japanese", "Kazakh", "Korean", "Lithuanian", "Latvian", "Burmese", "Nepali",
        "Dutch", "Romanian", "Russian", "Sinhala", "Turkish", "Vietnamese", "Chinese", "Afrikaans",
        "Azerbaijani", "Bengali", "Persian", "Hebrew", "Croatian", "Indonesian", "Georgian", "Khmer",
        "Macedonian", "Malayalam", "Mongolian", "Marathi", "Polish", "Pashto", "Portuguese", "Swedish",
        "Swahili", "Tamil", "Telugu", "Thai", "Tagalog", "Ukrainian", "Urdu", "Xhosa", "Galician",
        "Slovene"
    ]

    print("Available languages:")
    for language in languages:
        print(language)

    while True:
        language_choice = get_user_input("Choose a language for the output: ")
        if language_choice in languages:
            language_name = language_choice
            break
        else:
            print("Invalid choice. Please choose a valid language.")

    text_chunks = split_text_into_chunks(pdf_text)

    while True:
        main_choice = get_user_input("Choose an option: (1) Generate Course, (2) Generate Questions: ").lower()
        if main_choice not in ['1', '2']:
            print("Invalid choice. Please choose either '1' or '2'.")
            continue

        if main_choice == '1':
            prompt = create_prompts(pdf_text, "course")
            course_outline = generate_content(prompt)
            translated_course_outline = translate_text(course_outline, language_name)
            print("Course Outline and Lesson Plan:")
            print(translated_course_outline)
        else:
            previous_question_type = None
            while True:
                question_type = get_user_input("Choose the type of questions to generate (mcq, fill_in_the_blank, short_answer): ").lower()
                if question_type not in ['mcq', 'fill_in_the_blank', 'short_answer']:
                    print("Invalid choice. Please choose either 'mcq', 'fill_in_the_blank', or 'short_answer'.")
                    continue
                if question_type == previous_question_type:
                    print(f"You've already generated {question_type} questions. Please choose a different type.")
                    continue

                num_questions = int(get_user_input("Enter the number of questions to generate (5, 10, 15): "))
                if num_questions not in [5, 10, 15]:
                    print("Invalid number of questions. Please choose either 5, 10, or 15.")
                    continue

                questions = []
                for chunk in text_chunks:
                    prompt = create_prompts(chunk, question_type)
                    question = generate_content(prompt)
                    translated_question = translate_text(question, language_name)
                    questions.append(translated_question)

                print(f"{question_type.upper()} Questions:")
                for question in questions[:num_questions]:
                    print(question)
                    print()

                another_round = get_user_input("Do you want to generate a different type of questions? (yes/no): ").lower()
                if another_round != 'yes':
                    break
                previous_question_type = question_type

        another_main_round = get_user_input("Do you want to perform another operation? (yes/no): ").lower()
        if another_main_round != 'yes':
            break

# Start the chatbot
if __name__ == "__main__":
    chatbot()


# **3. URL is directed here for Translation**

In [ ]:
import PyPDF2
import google.generativeai as genai
from io import BytesIO
import requests
import multiprocessing
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import google.generativeai as genai
from google.colab import userdata
import warnings
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
import markdown
import time
import urllib.parse

warnings.filterwarnings("ignore")

# Configure the Google Generative AI API
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

def split_text_into_chunks(text, max_chunk_size=2000):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 > max_chunk_size:
            chunks.append(current_chunk)
            current_chunk = sentence
        else:
            if current_chunk:
                current_chunk += ". " + sentence
            else:
                current_chunk = sentence

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

def generate_content(prompt, max_tokens=1024, retries=3, wait=5):
    model = genai.GenerativeModel('gemini-pro')
    for attempt in range(retries):
        try:
            response = model.generate_content(prompt)
            if response.parts:
                return response.parts[0].text.strip()
            else:
                raise ValueError("No valid parts in response.")
        except Exception as e:
            print(f"Error: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {wait} seconds...")
                time.sleep(wait)
            else:
                return f"Failed to generate content after {retries} attempts."

def create_prompts(text, task_type):
    prompts = {
        "mcq": f"Read the following text carefully and generate multiple-choice questions. Each question should include:\n"
               f"1. A clear and concise question based on the text.\n"
               f"2. Four answer options (A, B, C, D), with one correct answer clearly indicated.\n"
               f"3. The questions should cover key concepts, definitions, critical points, and significant details discussed in the text.\n"
               f"4. Ensure the options are plausible and relevant to the content.\n\n"
               f"Text:\n{text}\n\nMCQ:",
        "fill_in_the_blank": f"Read the following text thoroughly and generate fill-in-the-blank questions. Each question should include:\n"
                            f"1. A sentence from the text with one key term or concept replaced by a blank.\n"
                            f"2. The correct term or concept that completes the sentence accurately.\n"
                            f"3. Focus on important information, such as key terms, dates, names, and concepts that are critical to understanding the text.\n\n"
                            f"Text:\n{text}\n\nFill in the blank:",
        "short_answer": f"Read the following text attentively and generate short answer questions. Each question should include:\n"
                        f"1. A clear and specific question that requires a brief response.\n"
                        f"2. The response should address key points, explanations, or definitions provided in the text.\n"
                        f"3. Ensure the questions encourage critical thinking and comprehension of the material, focusing on important details and concepts.\n\n"
                        f"Text:\n{text}\n\nShort answer question:",
        "course": f"Read the following text and generate a comprehensive, structured curriculum content. The content should include:\n"
                  f"1. Learning objectives and outcomes.\n"
                  f"2. Topic-wise breakdown with detailed descriptions.\n"
                  f"3. Key concepts, definitions, and explanations.\n"
                  f"4. Examples, illustrations, and case studies.\n"
                  f"5. Assessment and evaluation criteria.\n\n"
                  f"Text:\n{text}\n\nCurriculum Content:",
    }
    return prompts.get(task_type, "")

def get_user_input(prompt):
    return input(prompt)

def translate_text_google_translate(text, target_language):
    base_url = "https://translate.google.com/?sl=en&tl={}&text={}&op=translate"
    encoded_text = urllib.parse.quote(text)
    url = base_url.format(target_language, encoded_text)
    print(f"Please use the following URL to view the translation: {url}")

def chatbot():
    pdf_path = get_user_input("Enter the path to your PDF file: ")
    pdf_text = extract_text_from_pdf(pdf_path)

    text_chunks = split_text_into_chunks(pdf_text)

    while True:
        main_choice = get_user_input("Choose an option: (1) Generate Course, (2) Generate Questions: ").lower()
        if main_choice not in ['1', '2']:
            print("Invalid choice. Please choose either '1' or '2'.")
            continue

        if main_choice == '1':
            prompt = create_prompts(pdf_text, "course")
            course_outline = generate_content(prompt)
            print("Course Outline and Lesson Plan (in English):")
            print(course_outline)
            translate_choice = get_user_input("Do you want to translate the course outline? (yes/no): ").lower()
            if translate_choice == 'yes':
                languages = {
                    "Afrikaans": "af", "Albanian": "sq", "Amharic": "am", "Arabic": "ar", "Armenian": "hy", "Azerbaijani": "az",
                    "Basque": "eu", "Belarusian": "be", "Bengali": "bn", "Bosnian": "bs", "Bulgarian": "bg", "Catalan": "ca",
                    "Cebuano": "ceb", "Chichewa": "ny", "Chinese Simplified": "zh-cn", "Chinese Traditional": "zh-tw",
                    "Croatian": "hr", "Czech": "cs", "Danish": "da", "Dutch": "nl", "English": "en", "Esperanto": "eo",
                    "Estonian": "et", "Filipino": "tl", "Finnish": "fi", "French": "fr", "Galician": "gl", "Georgian": "ka",
                    "German": "de", "Greek": "el", "Gujarati": "gu", "Haitian Creole": "ht", "Hausa": "ha", "Hawaiian": "haw",
                    "Hebrew": "he", "Hindi": "hi", "Hmong": "hmn", "Hungarian": "hu", "Icelandic": "is", "Igbo": "ig",
                    "Indonesian": "id", "Irish": "ga", "Italian": "it", "Japanese": "ja", "Javanese": "jw", "Kannada": "kn",
                    "Kazakh": "kk", "Khmer": "km", "Korean": "ko", "Kurdish": "ku", "Kyrgyz": "ky", "Lao": "lo", "Latin": "la",
                    "Latvian": "lv", "Lithuanian": "lt", "Luxembourgish": "lb", "Macedonian": "mk", "Malagasy": "mg",
                    "Malay": "ms", "Malayalam": "ml", "Maltese": "mt", "Maori": "mi", "Marathi": "mr", "Mongolian": "mn",
                    "Myanmar (Burmese)": "my", "Nepali": "ne", "Norwegian": "no", "Pashto": "ps", "Persian": "fa", "Polish": "pl",
                    "Portuguese": "pt", "Punjabi": "pa", "Romanian": "ro", "Russian": "ru", "Samoan": "sm", "Scots Gaelic": "gd",
                    "Serbian": "sr", "Sesotho": "st", "Shona": "sn", "Sindhi": "sd", "Sinhala": "si", "Slovak": "sk",
                    "Slovenian": "sl", "Somali": "so", "Spanish": "es", "Sundanese": "su", "Swahili": "sw", "Swedish": "sv",
                    "Tajik": "tg", "Tamil": "ta", "Telugu": "te", "Thai": "th", "Turkish": "tr", "Ukrainian": "uk", "Urdu": "ur",
                    "Uzbek": "uz", "Vietnamese": "vi", "Welsh": "cy", "Xhosa": "xh", "Yiddish": "yi", "Yoruba": "yo", "Zulu": "zu"
                }
                print("Available languages:")
                for language, code in languages.items():
                    print(f"{language} ({code})")
                language_choice = get_user_input("Enter the language code you want to translate to: ")
                if language_choice in languages.values():
                    translate_text_google_translate(course_outline, language_choice)
                else:
                    print("Invalid language code.")

        else:
            previous_question_type = None
            while True:
                question_type = get_user_input("Choose the type of questions to generate (mcq, fill_in_the_blank, short_answer): ").lower()
                if question_type not in ['mcq', 'fill_in_the_blank', 'short_answer']:
                    print("Invalid choice. Please choose either 'mcq', 'fill_in_the_blank', or 'short_answer'.")
                    continue
                if question_type == previous_question_type:
                    print(f"You've already generated {question_type} questions. Please choose a different type.")
                    continue

                num_questions = int(get_user_input("Enter the number of questions to generate (5, 10, 15): "))
                if num_questions not in [5, 10, 15]:
                    print("Invalid number of questions. Please choose either 5, 10, or 15.")
                    continue

                questions = []
                for chunk in text_chunks:
                    prompt = create_prompts(chunk, question_type)
                    question = generate_content(prompt)
                    questions.append(question)

                print(f"{question_type.upper()} Questions (in English):")
                for question in questions[:num_questions]:
                    print(question)
                    print()

                translate_choice = get_user_input("Do you want to translate the questions? (yes/no): ").lower()
                if translate_choice == 'yes':
                    languages = {
                        "Afrikaans": "af", "Albanian": "sq", "Amharic": "am", "Arabic": "ar", "Armenian": "hy", "Azerbaijani": "az",
                        "Basque": "eu", "Belarusian": "be", "Bengali": "bn", "Bosnian": "bs", "Bulgarian": "bg", "Catalan": "ca",
                        "Cebuano": "ceb", "Chichewa": "ny", "Chinese Simplified": "zh-cn", "Chinese Traditional": "zh-tw",
                        "Croatian": "hr", "Czech": "cs", "Danish": "da", "Dutch": "nl", "English": "en", "Esperanto": "eo",
                        "Estonian": "et", "Filipino": "tl", "Finnish": "fi", "French": "fr", "Galician": "gl", "Georgian": "ka",
                        "German": "de", "Greek": "el", "Gujarati": "gu", "Haitian Creole": "ht", "Hausa": "ha", "Hawaiian": "haw",
                        "Hebrew": "he", "Hindi": "hi", "Hmong": "hmn", "Hungarian": "hu", "Icelandic": "is", "Igbo": "ig",
                        "Indonesian": "id", "Irish": "ga", "Italian": "it", "Japanese": "ja", "Javanese": "jw", "Kannada": "kn",
                        "Kazakh": "kk", "Khmer": "km", "Korean": "ko", "Kurdish": "ku", "Kyrgyz": "ky", "Lao": "lo", "Latin": "la",
                        "Latvian": "lv", "Lithuanian": "lt", "Luxembourgish": "lb", "Macedonian": "mk", "Malagasy": "mg",
                        "Malay": "ms", "Malayalam": "ml", "Maltese": "mt", "Maori": "mi", "Marathi": "mr", "Mongolian": "mn",
                        "Myanmar (Burmese)": "my", "Nepali": "ne", "Norwegian": "no", "Pashto": "ps", "Persian": "fa", "Polish": "pl",
                        "Portuguese": "pt", "Punjabi": "pa", "Romanian": "ro", "Russian": "ru", "Samoan": "sm", "Scots Gaelic": "gd",
                        "Serbian": "sr", "Sesotho": "st", "Shona": "sn", "Sindhi": "sd", "Sinhala": "si", "Slovak": "sk",
                        "Slovenian": "sl", "Somali": "so", "Spanish": "es", "Sundanese": "su", "Swahili": "sw", "Swedish": "sv",
                        "Tajik": "tg", "Tamil": "ta", "Telugu": "te", "Thai": "th", "Turkish": "tr", "Ukrainian": "uk", "Urdu": "ur",
                        "Uzbek": "uz", "Vietnamese": "vi", "Welsh": "cy", "Xhosa": "xh", "Yiddish": "yi", "Yoruba": "yo", "Zulu": "zu"
                    }
                    print("Available languages:")
                    for language, code in languages.items():
                        print(f"{language} ({code})")
                    language_choice = get_user_input("Enter the language code you want to translate to: ")
                    if language_choice in languages.values():
                        for question in questions[:num_questions]:
                            translate_text_google_translate(question, language_choice)
                    else:
                        print("Invalid language code.")

                another_round = get_user_input("Do you want to generate a different type of questions? (yes/no): ").lower()
                if another_round != 'yes':
                    break
                previous_question_type = question_type

        another_main_round = get_user_input("Do you want to perform another operation? (yes/no): ").lower()
        if another_main_round != 'yes':
            break

# Start the chatbot
if __name__ == "__main__":
    chatbot()


# **4. Display using Googletrans library**

In [ ]:
import PyPDF2
import google.generativeai as genai
from io import BytesIO
import requests
import multiprocessing
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import google.generativeai as genai
from google.colab import userdata
import warnings
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
import markdown
import time
import urllib.parse
from googletrans import Translator, LANGUAGES

warnings.filterwarnings("ignore")

# Configure the Google Generative AI API
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

def split_text_into_chunks(text, max_chunk_size=2000):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 > max_chunk_size:
            chunks.append(current_chunk)
            current_chunk = sentence
        else:
            if current_chunk:
                current_chunk += ". " + sentence
            else:
                current_chunk = sentence

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

def generate_content(prompt, max_tokens=1024, retries=3, wait=5):
    model = genai.GenerativeModel('gemini-pro')
    for attempt in range(retries):
        try:
            response = model.generate_content(prompt)
            if response.parts:
                return response.parts[0].text.strip()
            else:
                raise ValueError("No valid parts in response.")
        except Exception as e:
            print(f"Error: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {wait} seconds...")
                time.sleep(wait)
            else:
                return f"Failed to generate content after {retries} attempts."

def create_prompts(text, task_type):
    prompts = {
        "mcq": f"Read the following text carefully and generate multiple-choice questions. Each question should include:\n"
               f"1. A clear and concise question based on the text.\n"
               f"2. Four answer options (A, B, C, D), with one correct answer clearly indicated.\n"
               f"3. The questions should cover key concepts, definitions, critical points, and significant details discussed in the text.\n"
               f"4. Ensure the options are plausible and relevant to the content.\n\n"
               f"Text:\n{text}\n\nMCQ:",
        "fill_in_the_blank": f"Read the following text thoroughly and generate fill-in-the-blank questions. Each question should include:\n"
                            f"1. A sentence from the text with one key term or concept replaced by a blank.\n"
                            f"2. The correct term or concept that completes the sentence accurately.\n"
                            f"3. Focus on important information, such as key terms, dates, names, and concepts that are critical to understanding the text.\n\n"
                            f"Text:\n{text}\n\nFill in the blank:",
        "short_answer": f"Read the following text attentively and generate short answer questions. Each question should include:\n"
                        f"1. A clear and specific question that requires a brief response.\n"
                        f"2. The response should address key points, explanations, or definitions provided in the text.\n"
                        f"3. Ensure the questions encourage critical thinking and comprehension of the material, focusing on important details and concepts.\n\n"
                        f"Text:\n{text}\n\nShort answer question:",
        "course": f"Read the following text and generate a comprehensive, structured curriculum content. The content should include:\n"
                  f"1. Learning objectives and outcomes.\n"
                  f"2. Topic-wise breakdown with detailed descriptions.\n"
                  f"3. Key concepts, definitions, and explanations.\n"
                  f"4. Examples, illustrations, and case studies.\n"
                  f"5. Assessment and evaluation criteria.\n\n"
                  f"Text:\n{text}\n\nCurriculum Content:",
    }
    return prompts.get(task_type, "")

def get_user_input(prompt):
    return input(prompt)

def translate_text_google_translate(text, target_language):
    translator = Translator()
    translation = translator.translate(text, dest=target_language)
    return translation.text

def chatbot():
    pdf_path = get_user_input("Enter the path to your PDF file: ")
    pdf_text = extract_text_from_pdf(pdf_path)

    text_chunks = split_text_into_chunks(pdf_text)

    while True:
        main_choice = get_user_input("Choose an option: (1) Generate Course, (2) Generate Questions: ").lower()
        if main_choice not in ['1', '2']:
            print("Invalid choice. Please choose either '1' or '2'.")
            continue

        if main_choice == '1':
            prompt = create_prompts(pdf_text, "course")
            course_outline = generate_content(prompt)
            print("Course Outline and Lesson Plan (in English):")
            print(course_outline)
            translate_choice = get_user_input("Do you want to translate the course outline? (yes/no): ").lower()
            if translate_choice == 'yes':
                languages = {v: k for k, v in LANGUAGES.items()}  # Reverse lookup
                print("Available languages:")
                for code, language in languages.items():
                    print(f"{language} ({code})")
                language_choice = get_user_input("Enter the language code you want to translate to: ")
                if language_choice in languages:
                    translated_outline = translate_text_google_translate(course_outline, language_choice)
                    print(f"Translated Course Outline ({languages[language_choice]}):")
                    print(translated_outline)
                else:
                    print("Invalid language code.")

        else:
            previous_question_type = None
            while True:
                question_type = get_user_input("Choose the type of questions to generate (mcq, fill_in_the_blank, short_answer): ").lower()
                if question_type not in ['mcq', 'fill_in_the_blank', 'short_answer']:
                    print("Invalid choice. Please choose either 'mcq', 'fill_in_the_blank', or 'short_answer'.")
                    continue
                if question_type == previous_question_type:
                    print(f"You've already generated {question_type} questions. Please choose a different type.")
                    continue

                num_questions = int(get_user_input("Enter the number of questions to generate (5, 10, 15): "))
                if num_questions not in [5, 10, 15]:
                    print("Invalid number of questions. Please choose either 5, 10, or 15.")
                    continue

                questions = []
                for chunk in text_chunks:
                    prompt = create_prompts(chunk, question_type)
                    question = generate_content(prompt)
                    questions.append(question)

                print(f"{question_type.upper()} Questions (in English):")
                for question in questions[:num_questions]:
                    print(question)
                    print()

                translate_choice = get_user_input("Do you want to translate the questions? (yes/no): ").lower()
                if translate_choice == 'yes':
                    languages = {v: k for k, v in LANGUAGES.items()}  # Reverse lookup
                    print("Available languages:")
                    for code, language in languages.items():
                        print(f"{language} ({code})")
                    language_choice = get_user_input("Enter the language code you want to translate to: ")
                    if language_choice in languages:
                        translated_questions = []
                        for question in questions[:num_questions]:
                            translated_question = translate_text_google_translate(question, language_choice)
                            translated_questions.append(translated_question)
                            print(f"Translated {question_type.upper()} Question ({languages[language_choice]}):")
                            print(translated_question)
                        questions = translated_questions
                    else:
                        print("Invalid language code.")

                another_round = get_user_input("Do you want to generate a different type of questions? (yes/no): ").lower()
                if another_round != 'yes':
                    break
                previous_question_type = question_type

        another_main_round = get_user_input("Do you want to perform another operation? (yes/no): ").lower()
        if another_main_round != 'yes':
            break

# Start the chatbot
if __name__ == "__main__":
    chatbot()


Enter the path to your PDF file: /content/football_tutorial.pdf
Choose an option: (1) Generate Course, (2) Generate Questions: 2
Choose the type of questions to generate (mcq, fill_in_the_blank, short_answer): mcq
Enter the number of questions to generate (5, 10, 15): 5


ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1140.67ms


MCQ Questions (in English):
**Question 1:**

Which of the following is the primary focus of football?

(A) Strength and agility
(B) Foot speed and ball control
(C) Endurance and strategy
(D) Teamwork and communication

**Answer: B**

**Question 2:**

What is the intended audience for this tutorial?

(A) Experienced football players
(B) Individuals with little knowledge of football
(C) Coaches and referees
(D) Fans of a specific football team

**Answer: B**

**Question 3:**

What is NOT included in the scope of this tutorial?

(A) Basic rules of the game
(B) Profiles of famous players
(C) Comprehensive guide to playing the game
(D) Information on football associations

**Answer: C**

**Question 4:**

What is a requirement for using the information in the tutorial?

(A) A subscription to Tutorials Point
(B) A written agreement from the publisher
(C) A passion for football and a desire to learn
(D) No requirements are mentioned

**Answer: C**

**Question 5:**

According to the disclaimer,

# **5. JSON Storing and Retrieval**

In [ ]:
import PyPDF2
import google.generativeai as genai
from io import BytesIO
import requests
import multiprocessing
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import google.generativeai as genai
from google.colab import userdata
import warnings
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
import markdown
import time
import urllib.parse
from googletrans import Translator, LANGUAGES
import json
import os

warnings.filterwarnings("ignore")

# Configure the Google Generative AI API
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

def split_text_into_chunks(text, max_chunk_size=2000):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 > max_chunk_size:
            chunks.append(current_chunk)
            current_chunk = sentence
        else:
            if current_chunk:
                current_chunk += ". " + sentence
            else:
                current_chunk = sentence

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

def generate_content(prompt, max_tokens=1024, retries=3, wait=5):
    model = genai.GenerativeModel('gemini-pro')
    for attempt in range(retries):
        try:
            response = model.generate_content(prompt)
            if response.parts:
                return response.parts[0].text.strip()
            else:
                raise ValueError("No valid parts in response.")
        except Exception as e:
            print(f"Error: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {wait} seconds...")
                time.sleep(wait)
            else:
                return f"Failed to generate content after {retries} attempts."

def create_prompts(text, task_type):
    prompts = {
        "mcq": f"Read the following text carefully and generate multiple-choice questions. Each question should include:\n"
               f"1. A clear and concise question based on the text.\n"
               f"2. Four answer options (A, B, C, D), with one correct answer clearly indicated.\n"
               f"3. The questions should cover key concepts, definitions, critical points, and significant details discussed in the text.\n"
               f"4. Ensure the options are plausible and relevant to the content.\n\n"
               f"Text:\n{text}\n\nMCQ:",
        "fill_in_the_blank": f"Read the following text thoroughly and generate fill-in-the-blank questions. Each question should include:\n"
                            f"1. A sentence from the text with one key term or concept replaced by a blank.\n"
                            f"2. The correct term or concept that completes the sentence accurately.\n"
                            f"3. Focus on important information, such as key terms, dates, names, and concepts that are critical to understanding the text.\n\n"
                            f"Text:\n{text}\n\nFill in the blank:",
        "short_answer": f"Read the following text attentively and generate short answer questions. Each question should include:\n"
                        f"1. A clear and specific question that requires a brief response.\n"
                        f"2. The response should address key points, explanations, or definitions provided in the text.\n"
                        f"3. Ensure the questions encourage critical thinking and comprehension of the material, focusing on important details and concepts.\n\n"
                        f"Text:\n{text}\n\nShort answer question:",
        "course": f"Read the following text and generate a comprehensive, structured curriculum content. The content should include:\n"
                  f"1. Learning objectives and outcomes.\n"
                  f"2. Topic-wise breakdown with detailed descriptions.\n"
                  f"3. Key concepts, definitions, and explanations.\n"
                  f"4. Examples, illustrations, and case studies.\n"
                  f"5. Assessment and evaluation criteria.\n\n"
                  f"Text:\n{text}\n\nCurriculum Content:",
    }
    return prompts.get(task_type, "")

def get_user_input(prompt):
    return input(prompt)

def translate_text_google_translate(text, target_language):
    translator = Translator()
    translation = translator.translate(text, dest=target_language)
    return translation.text

def save_to_json(data, filename):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)

def load_from_json(filename):
    if os.path.exists(filename):
        with open(filename, 'r') as file:
            return json.load(file)
    return {}

def chatbot():
    pdf_path = get_user_input("Enter the path to your PDF file: ")
    pdf_text = extract_text_from_pdf(pdf_path)

    text_chunks = split_text_into_chunks(pdf_text)
    data = load_from_json('generated_content.json')

    while True:
        main_choice = get_user_input("Choose an option: (1) Generate Course, (2) Generate Questions: ").lower()
        if main_choice not in ['1', '2']:
            print("Invalid choice. Please choose either '1' or '2'.")
            continue

        if main_choice == '1':
            prompt = create_prompts(pdf_text, "course")
            course_outline = generate_content(prompt)
            print("Course Outline and Lesson Plan (in English):")
            print(course_outline)
            data['course_outline'] = course_outline
            translate_choice = get_user_input("Do you want to translate the course outline? (yes/no): ").lower()
            if translate_choice == 'yes':
                languages = {v: k for k, v in LANGUAGES.items()}  # Reverse lookup
                print("Available languages:")
                for code, language in languages.items():
                    print(f"{language} ({code})")
                language_choice = get_user_input("Enter the language code you want to translate to: ")
                if language_choice in languages:
                    translated_outline = translate_text_google_translate(course_outline, language_choice)
                    print(f"Translated Course Outline ({languages[language_choice]}):")
                    print(translated_outline)
                    data['translated_course_outline'] = {language_choice: translated_outline}
                else:
                    print("Invalid language code.")

        else:
            previous_question_type = None
            while True:
                question_type = get_user_input("Choose the type of questions to generate (mcq, fill_in_the_blank, short_answer): ").lower()
                if question_type not in ['mcq', 'fill_in_the_blank', 'short_answer']:
                    print("Invalid choice. Please choose either 'mcq', 'fill_in_the_blank', or 'short_answer'.")
                    continue
                if question_type == previous_question_type:
                    print(f"You've already generated {question_type} questions. Please choose a different type.")
                    continue

                num_questions = int(get_user_input("Enter the number of questions to generate (5, 10, 15): "))
                if num_questions not in [5, 10, 15]:
                    print("Invalid number of questions. Please choose either 5, 10, or 15.")
                    continue

                questions = []
                for chunk in text_chunks:
                    prompt = create_prompts(chunk, question_type)
                    question = generate_content(prompt)
                    questions.append(question)

                print(f"{question_type.upper()} Questions (in English):")
                for question in questions[:num_questions]:
                    print(question)
                    print()

                data[question_type] = questions[:num_questions]

                translate_choice = get_user_input("Do you want to translate the questions? (yes/no): ").lower()
                if translate_choice == 'yes':
                    languages = {v: k for k, v in LANGUAGES.items()}  # Reverse lookup
                    print("Available languages:")
                    for code, language in languages.items():
                        print(f"{language} ({code})")
                    language_choice = get_user_input("Enter the language code you want to translate to: ")
                    if language_choice in languages:
                        translated_questions = []
                        for question in questions[:num_questions]:
                            translated_question = translate_text_google_translate(question, language_choice)
                            translated_questions.append(translated_question)
                            print(f"Translated {question_type.upper()} Question ({languages[language_choice]}):")
                            print(translated_question)
                        data[f'translated_{question_type}'] = {language_choice: translated_questions}
                    else:
                        print("Invalid language code.")

                another_round = get_user_input("Do you want to generate a different type of questions? (yes/no): ").lower()
                if another_round != 'yes':
                    break
                previous_question_type = question_type

        save_to_json(data, 'generated_content.json')

        another_main_round = get_user_input("Do you want to perform another operation? (yes/no): ").lower()
        if another_main_round != 'yes':
            break

# Start the chatbot
if __name__ == "__main__":
    chatbot()


Enter the path to your PDF file: /content/football_tutorial.pdf
Choose an option: (1) Generate Course, (2) Generate Questions: 2
Choose the type of questions to generate (mcq, fill_in_the_blank, short_answer): mcq
Enter the number of questions to generate (5, 10, 15): 10
MCQ Questions (in English):
**Question 1:**

What is the key requirement for playing football, as mentioned in the text?

(A) Intelligence
(B) Stamina
(C) Strength
(D) Speed

**Correct Answer: B**

**Question 2:**

According to the tutorial, who is the intended audience?

(A) Professional football players
(B) Football experts
(C) People unfamiliar with football
(D) Sports enthusiasts

**Correct Answer: C**

**Question 3:**

What is the main purpose of the tutorial?

(A) To teach how to play football
(B) To explain the fundamental rules of football
(C) To provide details about football associations
(D) To highlight prominent football players

**Correct Answer: B**

**Question 4:**

Which of the following is NOT a requir

# **6. JSON Testing**

In [ ]:
import PyPDF2
import google.generativeai as genai
import json
from io import BytesIO
import requests
import multiprocessing
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import google.generativeai as genai
from google.colab import userdata
import warnings
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
import markdown
import time
import urllib.parse
from googletrans import Translator, LANGUAGES
import os

warnings.filterwarnings("ignore")

# Configure the Google Generative AI API
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

def split_text_into_chunks(text, max_chunk_size=2000):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 > max_chunk_size:
            chunks.append(current_chunk)
            current_chunk = sentence
        else:
            if current_chunk:
                current_chunk += ". " + sentence
            else:
                current_chunk = sentence

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

def generate_content(prompt, max_tokens=1024, retries=3, wait=5):
    model = genai.GenerativeModel('gemini-pro')
    for attempt in range(retries):
        try:
            response = model.generate_content(prompt)
            if response.parts:
                return response.parts[0].text.strip()
            else:
                raise ValueError("No valid parts in response.")
        except Exception as e:
            print(f"Error: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {wait} seconds...")
                time.sleep(wait)
            else:
                return f"Failed to generate content after {retries} attempts."

def create_prompts(text, task_type):
    prompts = {
        "mcq": f"Read the following text carefully and generate multiple-choice questions. Each question should include:\n"
               f"1. A clear and concise question based on the text.\n"
               f"2. Four answer options (A, B, C, D), with one correct answer clearly indicated.\n"
               f"3. The questions should cover key concepts, definitions, critical points, and significant details discussed in the text.\n"
               f"4. Ensure the options are plausible and relevant to the content.\n\n"
               f"Text:\n{text}\n\nMCQ:",
        "fill_in_the_blank": f"Read the following text thoroughly and generate fill-in-the-blank questions. Each question should include:\n"
                            f"1. A sentence from the text with one key term or concept replaced by a blank.\n"
                            f"2. The correct term or concept that completes the sentence accurately.\n"
                            f"3. Focus on important information, such as key terms, dates, names, and concepts that are critical to understanding the text.\n\n"
                            f"Text:\n{text}\n\nFill in the blank:",
        "short_answer": f"Read the following text attentively and generate short answer questions. Each question should include:\n"
                        f"1. A clear and specific question that requires a brief response.\n"
                        f"2. The response should address key points, explanations, or definitions provided in the text.\n"
                        f"3. Ensure the questions encourage critical thinking and comprehension of the material, focusing on important details and concepts.\n\n"
                        f"Text:\n{text}\n\nShort answer question:",
        "course": f"Read the following text and generate a comprehensive, structured curriculum content. The content should include:\n"
                  f"1. Learning objectives and outcomes.\n"
                  f"2. Topic-wise breakdown with detailed descriptions.\n"
                  f"3. Key concepts, definitions, and explanations.\n"
                  f"4. Examples, illustrations, and case studies.\n"
                  f"5. Assessment and evaluation criteria.\n\n"
                  f"Text:\n{text}\n\nCurriculum Content:",
    }
    return prompts.get(task_type, "")

def get_user_input(prompt):
    return input(prompt)

def translate_text_google_translate(text, target_language):
    translator = Translator()
    translation = translator.translate(text, dest=target_language)
    return translation.text

def save_to_json(filename, data):
    with open(filename, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

def load_from_json(filename):
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as json_file:
            return json.load(json_file)
    return {}

def attempt_questions(filename):
    questions_data = load_from_json(filename)
    if not questions_data:
        print("No questions available. Please generate questions first.")
        return

    for q_type, questions in questions_data.items():
        print(f"Attempting {q_type.upper()} Questions:")
        for question in questions:
            print(question)
            user_answer = get_user_input("Your answer: ")
            # In a real scenario, here you would compare user_answer with the correct answer
            print(f"Your answer: {user_answer}\n")

def chatbot():
    pdf_path = get_user_input("Enter the path to your PDF file: ")
    pdf_text = extract_text_from_pdf(pdf_path)

    text_chunks = split_text_into_chunks(pdf_text)

    while True:
        main_choice = get_user_input("Choose an option: (1) Generate Course, (2) Generate Questions, (3) Attempt Questions: ").lower()
        if main_choice not in ['1', '2', '3']:
            print("Invalid choice. Please choose '1', '2', or '3'.")
            continue

        if main_choice == '1':
            prompt = create_prompts(pdf_text, "course")
            course_outline = generate_content(prompt)
            print("Course Outline and Lesson Plan (in English):")
            print(course_outline)
            save_to_json('course_outline.json', {"course_outline": course_outline})

            translate_choice = get_user_input("Do you want to translate the course outline? (yes/no): ").lower()
            if translate_choice == 'yes':
                languages = {v: k for k, v in LANGUAGES.items()}  # Reverse lookup
                print("Available languages:")
                for code, language in languages.items():
                    print(f"{language} ({code})")
                language_choice = get_user_input("Enter the language code you want to translate to: ")
                if language_choice in languages:
                    translated_outline = translate_text_google_translate(course_outline, language_choice)
                    print(f"Translated Course Outline ({languages[language_choice]}):")
                    print(translated_outline)
                    save_to_json(f'course_outline_{language_choice}.json', {"course_outline": translated_outline})
                else:
                    print("Invalid language code.")

        elif main_choice == '2':
            previous_question_type = None
            all_questions = load_from_json('questions.json')
            while True:
                question_type = get_user_input("Choose the type of questions to generate (mcq, fill_in_the_blank, short_answer): ").lower()
                if question_type not in ['mcq', 'fill_in_the_blank', 'short_answer']:
                    print("Invalid choice. Please choose either 'mcq', 'fill_in_the_blank', or 'short_answer'.")
                    continue
                if question_type == previous_question_type:
                    print(f"You've already generated {question_type} questions. Please choose a different type.")
                    continue

                num_questions = int(get_user_input("Enter the number of questions to generate (5, 10, 15): "))
                if num_questions not in [5, 10, 15]:
                    print("Invalid number of questions. Please choose either 5, 10, or 15.")
                    continue

                questions = []
                for chunk in text_chunks:
                    prompt = create_prompts(chunk, question_type)
                    question = generate_content(prompt)
                    questions.append(question)

                all_questions[question_type] = questions[:num_questions]
                save_to_json('questions.json', all_questions)

                print(f"{question_type.upper()} Questions (in English):")
                for question in questions[:num_questions]:
                    print(question)
                    print()

                translate_choice = get_user_input("Do you want to translate the questions? (yes/no): ").lower()
                if translate_choice == 'yes':
                    languages = {v: k for k, v in LANGUAGES.items()}  # Reverse lookup
                    print("Available languages:")
                    for code, language in languages.items():
                        print(f"{language} ({code})")
                    language_choice = get_user_input("Enter the language code you want to translate to: ")
                    if language_choice in languages:
                        translated_questions = []
                        for question in questions[:num_questions]:
                            translated_question = translate_text_google_translate(question, language_choice)

                            translated_questions.append(translated_question)
                            print(f"Translated {question_type.upper()} Question ({languages[language_choice]}):")
                            print(translated_question)
                        all_questions[f"{question_type}_{language_choice}"] = translated_questions
                        save_to_json('questions.json', all_questions)
                    else:
                        print("Invalid language code.")

                another_round = get_user_input("Do you want to generate a different type of questions? (yes/no): ").lower()
                if another_round != 'yes':
                    break
                previous_question_type = question_type

        elif main_choice == '3':
            attempt_questions('questions.json')

        another_main_round = get_user_input("Do you want to perform another operation? (yes/no): ").lower()
        if another_main_round != 'yes':
            break

# Start the chatbot
if __name__ == "__main__":
    chatbot()




Enter the path to your PDF file: /content/football_tutorial.pdf
Choose an option: (1) Generate Course, (2) Generate Questions, (3) Attempt Questions: 2
Choose the type of questions to generate (mcq, fill_in_the_blank, short_answer): mcq
Enter the number of questions to generate (5, 10, 15): 10
MCQ Questions (in English):
**Question 1:**

What is the primary focus of the football tutorial?

(A) Providing advanced techniques for playing the game
(B) Explaining basic rules and terminology for beginners
(C) Discussing historical events and prominent players
(D) Organizing tournaments around the world

**Answer: B**

**Question 2:**

Who is the intended audience for this tutorial?

(A) Professional footballers
(B) Those who want to learn the basics of football
(C) Sports enthusiasts
(D) People with prior experience in football

**Answer: B**

**Question 3:**

What is a key requirement for using this tutorial?

(A) Comprehensive knowledge of football
(B) A desire to win at all costs
(C) A pa

# **7. Course Outline and Talking to PDF**

In [ ]:
import PyPDF2
import google.generativeai as genai
import json
from io import BytesIO
import requests
import multiprocessing
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import google.generativeai as genai
from google.colab import userdata
import warnings
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
import markdown
import time
import urllib.parse
from googletrans import Translator, LANGUAGES
import os

warnings.filterwarnings("ignore")

# Configure the Google Generative AI API
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

def split_text_into_chunks(text, max_chunk_size=2000):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 > max_chunk_size:
            chunks.append(current_chunk)
            current_chunk = sentence
        else:
            if current_chunk:
                current_chunk += ". " + sentence
            else:
                current_chunk = sentence

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

def generate_content(prompt, max_tokens=1024, retries=3, wait=5):
    model = genai.GenerativeModel('gemini-pro')
    for attempt in range(retries):
        try:
            response = model.generate_content(prompt)
            if response.parts:
                return response.parts[0].text.strip()
            else:
                raise ValueError("No valid parts in response.")
        except Exception as e:
            print(f"Error: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {wait} seconds...")
                time.sleep(wait)
            else:
                return f"Failed to generate content after {retries} attempts."

def create_prompts(text, task_type):
    prompts = {
        "mcq": f"Read the following text carefully and generate multiple-choice questions. Each question should include:\n"
               f"1. A clear and concise question based on the text.\n"
               f"2. Four answer options (A, B, C, D), with one correct answer clearly indicated.\n"
               f"3. The questions should cover key concepts, definitions, critical points, and significant details discussed in the text.\n"
               f"4. Ensure the options are plausible and relevant to the content.\n\n"
               f"Text:\n{text}\n\nMCQ:",
        "fill_in_the_blank": f"Read the following text thoroughly and generate fill-in-the-blank questions. Each question should include:\n"
                            f"1. A sentence from the text with one key term or concept replaced by a blank.\n"
                            f"2. The correct term or concept that completes the sentence accurately.\n"
                            f"3. Focus on important information, such as key terms, dates, names, and concepts that are critical to understanding the text.\n\n"
                            f"Text:\n{text}\n\nFill in the blank:",
        "short_answer": f"Read the following text attentively and generate short answer questions. Each question should include:\n"
                        f"1. A clear and specific question that requires a brief response.\n"
                        f"2. The response should address key points, explanations, or definitions provided in the text.\n"
                        f"3. Ensure the questions encourage critical thinking and comprehension of the material, focusing on important details and concepts.\n\n"
                        f"Text:\n{text}\n\nShort answer question:",
        "course": f"Read the following text and generate a comprehensive, structured course content. The content should include:\n"
                  f"1. Detailed learning objectives and outcomes for each chapter, helping students understand what they are expected to learn.\n"
                  f"2. A chapter-wise breakdown with detailed descriptions and subtopics, summarizing the main points for easy study.\n"
                  f"3. Key concepts, definitions, and explanations for each chapter, highlighting the essential information.\n"
                  f"4. Examples, illustrations, and case studies relevant to each chapter, to help students grasp practical applications.\n"
                  f"5. Practical exercises and activities for students to reinforce learning, ensuring they can apply what they have learned.\n"
                  f"6. Summaries and key takeaways for each chapter, so students can review the main points quickly.\n"
                  f"7. Supplementary resources and reading materials for further study, providing avenues for deeper exploration.\n"
                  f"8. Guidance notes and tips for teachers on how to effectively deliver the content and engage students.\n"
                  f"9. Incorporate interactive elements and multimedia content where possible, to enhance learning and retention.\n"
                  f"10. Ensure the curriculum is structured in a logical and progressive manner to facilitate a smooth learning experience.\n\n"
                  f"Text:\n{text}\n\nComprehensive Course Content:",
        "chat": f"Use the following text as a reference to answer questions based on its content.\n\n"
                f"Text:\n{text}\n\nQuestion:",
    }
    return prompts.get(task_type, "")



def get_user_input(prompt):
    return input(prompt)

def translate_text_google_translate(text, target_language):
    translator = Translator()
    translation = translator.translate(text, dest=target_language)
    return translation.text

def save_to_json(filename, data):
    with open(filename, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

def load_from_json(filename):
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as json_file:
            return json.load(json_file)
    return {}

def talk_to_pdf(text_chunks):
    print("You can now talk to the PDF. Ask any questions based on its content.")
    while True:
        user_question = get_user_input("Your question (type 'exit' to quit): ")
        if user_question.lower() == 'exit':
            break
        responses = []
        for chunk in text_chunks:
            prompt = create_prompts(chunk, "chat") + user_question
            response = generate_content(prompt)
            responses.append(response)
        print("Response:")
        print(" ".join(responses))
        print()

def chatbot():
    pdf_path = get_user_input("Enter the path to your PDF file: ")
    pdf_text = extract_text_from_pdf(pdf_path)

    text_chunks = split_text_into_chunks(pdf_text)

    while True:
        main_choice = get_user_input("Choose an option: (1) Generate Course, (2) Generate Questions, (3) Talk to the PDF: ").lower()
        if main_choice not in ['1', '2', '3']:
            print("Invalid choice. Please choose '1', '2', or '3'.")
            continue

        if main_choice == '1':
            prompt = create_prompts(pdf_text, "course")
            course_outline = generate_content(prompt)
            print("Course Outline and Lesson Plan (in English):")
            print(course_outline)
            save_to_json('course_outline.json', {"course_outline": course_outline})

            translate_choice = get_user_input("Do you want to translate the course outline? (yes/no): ").lower()
            if translate_choice == 'yes':
                languages = {v: k for k, v in LANGUAGES.items()}  # Reverse lookup
                print("Available languages:")
                for code, language in languages.items():
                    print(f"{language} ({code})")
                language_choice = get_user_input("Enter the language code you want to translate to: ")
                if language_choice in languages:
                    translated_outline = translate_text_google_translate(course_outline, language_choice)
                    print(f"Translated Course Outline ({languages[language_choice]}):")
                    print(translated_outline)
                    save_to_json(f'course_outline_{language_choice}.json', {"course_outline": translated_outline})
                else:
                    print("Invalid language code.")

        elif main_choice == '2':
            previous_question_type = None
            all_questions = load_from_json('questions.json')
            while True:
                question_type = get_user_input("Choose the type of questions to generate (mcq, fill_in_the_blank, short_answer): ").lower()
                if question_type not in ['mcq', 'fill_in_the_blank', 'short_answer']:
                    print("Invalid choice. Please choose either 'mcq', 'fill_in_the_blank', or 'short_answer'.")
                    continue
                if question_type == previous_question_type:
                    print(f"You've already generated {question_type} questions. Please choose a different type.")
                    continue

                num_questions = int(get_user_input("Enter the number of questions to generate (5, 10, 15): "))
                if num_questions not in [5, 10, 15]:
                    print("Invalid number of questions. Please choose either 5, 10, or 15.")
                    continue

                questions = []
                for chunk in text_chunks:
                    prompt = create_prompts(chunk, question_type)
                    question = generate_content(prompt)
                    questions.append(question)

                all_questions[question_type] = questions[:num_questions]
                save_to_json('questions.json', all_questions)

                print(f"{question_type.upper()} Questions (in English):")
                for question in questions[:num_questions]:
                    print(question)
                    print()

                translate_choice = get_user_input("Do you want to translate the questions? (yes/no): ").lower()
                if translate_choice == 'yes':
                    languages = {v: k for k, v in LANGUAGES.items()}  # Reverse lookup
                    print("Available languages:")
                    for code, language in languages.items():
                        print(f"{language} ({code})")
                    language_choice = get_user_input("Enter the language code you want to translate to: ")
                    if language_choice in languages:
                        translated_questions = []
                        for question in questions[:num_questions]:
                            translated_question = translate_text_google_translate(question, language_choice)

                            translated_questions.append(translated_question)
                            print(f"Translated {question_type.upper()} Question ({languages[language_choice]}):")
                            print(translated_question)
                        all_questions[f"{question_type}_{language_choice}"] = translated_questions
                        save_to_json('questions.json', all_questions)
                    else:
                        print("Invalid language code.")

                another_round = get_user_input("Do you want to generate a different type of questions? (yes/no): ").lower()
                if another_round != 'yes':
                    break
                previous_question_type = question_type

        elif main_choice == '3':
            talk_to_pdf(text_chunks)

        another_main_round = get_user_input("Do you want to perform another operation? (yes/no): ").lower()
        if another_main_round != 'yes':
            break

# Start the chatbot
if __name__ == "__main__":
    chatbot()


Enter the path to your PDF file: /content/new_resume.pdf
Choose an option: (1) Generate Course, (2) Generate Questions, (3) Talk to the PDF: 3
You can now talk to the PDF. Ask any questions based on its content.
Your question (type 'exit' to quit): what is this all about
Response:
Unfortunately, I do not have access to any reference text, so I am unable to answer this question from the provided context. This document is a resume of Abhishek Sinha, an AI engineer with experience in machine learning, deep learning, and cloud computing. He has worked on projects such as an AI-based multilingual course generator and has experience with Google Cloud, AWS, and Azure. His skills include prompt engineering, generative AI, working with LLM, API integration, AI systems, and more. He has a Bachelor of Engineering from Acharya Institute of Technology and an Executive Post Graduate Certification in Data Science & Artificial Intelligence from iHub, Divya Sampark IIT Roorkee.

Your question (type 'ex